In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import os
import re
import functools
from time import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import utils
import data_reader
from batch_norm import Batch_Normalizer
from model import CNN_Model

In [2]:
# File parameters
DATA_DIR = 'data'
TRAIN_IMAGES = 'data_train.bin'
TRAIN_TEMPLATES = 'fv_train.bin'
VALID_IMAGES = 'data_valid.bin'
VALID_TEMPLATES = 'fv_valid.bin'
TEST_IMAGES = 'data_test.bin'

# Data parameters
NUM_TRAIN_IMAGES = 100000
NUM_VALID_IMAGES = 10000
NUM_TEST_IMAGES = 10000
IMAGE_DIM = 48
TEMPLATE_DIM = 128

# Reader parameters
BATCH_SIZE = 100
VALID_BATCH_SIZE = 1000
NUM_READ_THREADS = 1
MIN_AFTER_DEQUEUE = 10000
CAPACITY = 11000

# Batch norm parameters
EMA_DECAY = 0.99
BN_EPSILON = 0.01

# Log parameters
LOG_DIR = 'logs'
SAVE_DIR = 'checkpoints'

In [3]:
def test_config2_dropout(dropout):

    model = CNN_Model('config2_dropout_avgpool_%.1f' % dropout, DATA_DIR, LOG_DIR, SAVE_DIR, 
                      IMAGE_DIM, TEMPLATE_DIM, TRAIN_IMAGES, TRAIN_TEMPLATES, VALID_IMAGES, 
                      VALID_TEMPLATES, NUM_TRAIN_IMAGES, NUM_VALID_IMAGES, EMA_DECAY, BN_EPSILON)

    model.add_conv('conv_1_1', ksize=[3, 3, 1, 10])
    model.add_batch_norm('bn_1_1')
    model.add_conv('conv_1_2', ksize=[3, 3, 10, 10])
    model.add_pool('max_pool_1', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_1_2')

    model.add_conv('conv_2_1', ksize=[3, 3, 10, 20])
    model.add_batch_norm('bn_2_1')
    model.add_conv('conv_2_2', ksize=[3, 3, 20, 20])
    model.add_pool('max_pool_2', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_2_2')

    model.add_conv('conv_3_1', ksize=[3, 3, 20, 40])
    model.add_batch_norm('bn_3_1')
    model.add_conv('conv_3_2', ksize=[3, 3, 40, 40])
    model.add_pool('max_pool_3', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_3_2')

    model.add_pool('avg_pool', 'avg', ksize=[1, 3, 3, 1], stride=[1, 3, 3, 1], dropout=dropout)
    model.add_fully_connected('fc', size=128)

    model.add_mse_loss('mse_loss')
    model.add_adam_optimizer('optimizer', init_learning_rate=0.01, decay=False)
    model.add_summaries('summaries')

    
    utils.add_description("""
    model = CNN_Model('config2_dropout_avgpool_%.1f', DATA_DIR, LOG_DIR, SAVE_DIR, 
                      IMAGE_DIM, TEMPLATE_DIM, TRAIN_IMAGES, TRAIN_TEMPLATES, VALID_IMAGES, 
                      VALID_TEMPLATES, NUM_TRAIN_IMAGES, NUM_VALID_IMAGES, EMA_DECAY, BN_EPSILON)

    model.add_conv('conv_1_1', ksize=[3, 3, 1, 10])
    model.add_batch_norm('bn_1_1')
    model.add_conv('conv_1_2', ksize=[3, 3, 10, 10])
    model.add_pool('max_pool_1', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_1_2')

    model.add_conv('conv_2_1', ksize=[3, 3, 10, 20])
    model.add_batch_norm('bn_2_1')
    model.add_conv('conv_2_2', ksize=[3, 3, 20, 20])
    model.add_pool('max_pool_2', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_2_2')

    model.add_conv('conv_3_1', ksize=[3, 3, 20, 40])
    model.add_batch_norm('bn_3_1')
    model.add_conv('conv_3_2', ksize=[3, 3, 40, 40])
    model.add_pool('max_pool_3', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_3_2')

    model.add_pool('avg_pool', 'avg', ksize=[1, 3, 3, 1], stride=[1, 3, 3, 1], dropout=%.1f)
    model.add_fully_connected('fc', size=128)

    model.add_mse_loss('mse_loss')
    model.add_adam_optimizer('optimizer', init_learning_rate=0.01, decay=False)
    model.add_summaries('summaries')
    
    total parameters = %d
    """  % (dropout, dropout, utils.num_parameters(model)), model.save_dir, model.name)
    
    
    model.initialize_session(restore=True)
    model.train(n_batches=10000, step_size=500, batch_size=100, valid_batch_size=1000, save=True)
    model.close_session()

In [4]:
for dropout in [0.1, 0.2, 0.3]:
    test_config2_dropout(dropout)

Description added at checkpoints\config2_dropout_avgpool_0.1\config2_dropout_avgpool_0.1_description.txt
Batch 500:
--> Train loss = 0.015912
--> Valid loss = 0.007862
Saving session checkpoints\config2_dropout_avgpool_0.1\model.ckpt-500
Batch 1000:
--> Train loss = 0.007636
--> Valid loss = 0.007706
Saving session checkpoints\config2_dropout_avgpool_0.1\model.ckpt-1000
Batch 1500:
--> Train loss = 0.007527
--> Valid loss = 0.007672
Saving session checkpoints\config2_dropout_avgpool_0.1\model.ckpt-1500
Batch 2000:
--> Train loss = 0.007459
--> Valid loss = 0.007628
Saving session checkpoints\config2_dropout_avgpool_0.1\model.ckpt-2000
Batch 2500:
--> Train loss = 0.007404
--> Valid loss = 0.007592
Saving session checkpoints\config2_dropout_avgpool_0.1\model.ckpt-2500
Batch 3000:
--> Train loss = 0.007346
--> Valid loss = 0.007576
Saving session checkpoints\config2_dropout_avgpool_0.1\model.ckpt-3000
Batch 3500:
--> Train loss = 0.007287
--> Valid loss = 0.007526
Saving session checkpoi